In [1]:
import cv2
import time
import datetime
import glob
import subprocess as sp
import numpy as np
import pandas as pd
import os

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#import opencv-python as cv
VIDEO_URL = "https://videos3.earthcam.com/fecnetwork/9974.flv/chunklist_w917803974.m3u8"
VIDEO_URL

'https://videos3.earthcam.com/fecnetwork/9974.flv/chunklist_w917803974.m3u8'

In [3]:
os.getcwd()
cam_data_dir = 'data/cam_data/'

In [4]:
ffmpgeg_dir = '../DSR/FFmpeg/bin/'
os.listdir(ffmpgeg_dir)

['ffmpeg.exe', 'ffplay.exe', 'ffprobe.exe']

In [5]:
cv2.namedWindow("Streetcam", cv2.WINDOW_NORMAL)
FFMPEG_BIN = ffmpgeg_dir + "ffmpeg"

command = [FFMPEG_BIN, "-i", VIDEO_URL,
           "-loglevel", "quiet",  # no text output
           "-an",   # disable audio
           "-f", "image2pipe",
           "-pix_fmt", "bgr24",
           "-vcodec", "rawvideo", "-"
           ]
pipe = sp.Popen(command, stdin = sp.PIPE, stdout = sp.PIPE)

while True:
    raw_image = pipe.stdout.read(1280*720*3) 
    image = np.fromstring(raw_image, dtype='uint8').reshape((720,1280,3))
    cv2.imshow("Streetcam",image)
    
    if cv2.waitKey(1) == 2:
        break

C:\ProgramData\Anaconda3\envs\DataScienceRetreat\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  from ipykernel import kernelapp as app


KeyboardInterrupt: 

In [ ]:
while True:
    raw_image = pipe.stdout.read(1280*720*3) 
    image = np.fromstring(raw_image, dtype='uint8').reshape((720,1280,3))
    cv2.imshow("Streetcam",image)

In [ ]:
files_new = set()
files_wr = set()

files = glob.glob(cam_data_dir + "*.jpg")
print(len(files),sorted(files))

In [ ]:
# -------- sorting paths in the txt file using the union
with open('cam_images.txt', 'w') as f:
    for item in sorted(files):
        f.write("%s\n" % item)

files_wr = files_wr.union(set(files))
print(files_wr)
print(type(files_wr))

def print_time(comment):
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime("%H:%M:%S.%f")[:-3]
    print(comment, st)

# Would like write all print outs to a log file - How :( ?
while True:
    # Run below code in while loop for actual normal frames || 16:44
    # raw_image = pipe.stdout.read(420*360*3) # read 432*240*3 bytes (= 1 frame)
    # image =  np.fromstring(raw_image, dtype='uint8').reshape((360,420,3))
    print("")
    print("Number "+str(counter))
    print_time("Beginning")
    raw_image = pipe.stdout.read(1280*720*3) # read 432*240*3 bytes (= 1 frame)
    print_time("reading image")
    print(raw_image is None)
    image = np.fromstring(raw_image, dtype='uint8').reshape((720,1280,3))
    print_time("showing image")
    cv2.imshow("Streetcam",image)
    print_time("")
    #sckit for img processing. s3 fs to plug it into cv2 for img writing.
    #Srcript Credit Pascal Schambach - Helped rewrite this
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('_%Y%m%d_%H%M%S')
    new_filename = 'capture' + st + '.jpg' #jpg smaller than >png (transparent)
    print(new_filename)
    #---
    print_time("write to path")
    cv2.imwrite(filepath.format(new_filename), image) #write to a filepath, by timestamp
    files = glob.glob("cam_data/*.jpg")
    new_files = set(files) - files_wr
    # -------- sorting paths in the txt file using the union
    print_time("Soting images")
    with open('cam_images.txt', 'a+') as f:
        for item in sorted(new_files):
            f.write("%s\n" % item)

    files_wr = files_wr.union(new_files)


    # What is happening here :( I get these glitched frames, must resolve
    # Use wait key 0 or ? This bit is a little confusing.
    print_time("wait key")
    if cv2.waitKey(1) == 2:
        break


    counter += 1

# This does not seem to work.

In [ ]:
cv2.destroyAllWindows()